# Final Project – Predicting Workout Duration from Early EndoMondo Fitness Data

Course: Machine Learning  
Dataset: EndoMondo / fitness tracking data with sequential heart-rate and speed measurements  
Team: Ayoob Al-Delaimy, Umar Khan, Hasaan Haq, Parwiz

---

## Project Overview

In this notebook, we build a complete machine learning pipeline using the EndoMondo fitness tracking dataset. The dataset contains sequential sensor data (heart rate, speed, GPS, etc.) for many workouts across different users and sports.

### Predictive Task (high level)

Goal: Predict total workout duration (in minutes) using only the first 5 minutes of sensor data (heart rate + speed) and basic context (sport, gender).

This reflects a realistic use case for fitness apps: can we estimate how long a workout will last, early in the session, using wearable sensor data?

### Notebook Structure (Rubric-aligned)

1. **Predictive Task & Evaluation Setup**  
2. **Exploratory Data Analysis (EDA) & Preprocessing**  
3. **Modeling (Baselines + ML Models)**  
4. **Evaluation (Metrics, Comparisons, Interpretation)**  
5. **Discussion of Related Work & Insights**

Each section includes well-commented code, clear visualizations, and explanatory markdown so the notebook is presentation-ready.


In [1]:
# ===========================
# Imports and global settings
# ===========================

import ast
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Plotting defaults for nicer visuals
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 1. Predictive Task & Evaluation Setup

### 1.1 Task Definition

The dataset contains **sequences of heart rate and speed** for individual workouts, along with metadata such as:

- `userId` – user identifier  
- `sport` – type of activity (e.g., running, cycling)  
- `gender` – optional demographic information  
- `heart_rate` – sequence of heart-rate measurements  
- `speed` or `derived_speed` – sequence of speed measurements  

From these sequences, we can derive:

- **Total workout duration** (in minutes) from the length of the time series  
- **Early-workout behavior**, such as the mean and variability of heart rate and speed during the first few minutes

> **Predictive Task:**  
> Given only the **first 5 minutes of heart-rate and speed** plus basic context (sport and gender), **predict the total workout duration (minutes)**.

This is a **regression** problem with a continuous output.

### 1.2 Real-world Motivation

In practice, fitness platforms (EndoMondo, Strava, etc.) often make decisions early in a workout:
- Suggesting routes of appropriate length  
- Adjusting training recommendations  
- Estimating finish time for pacing purposes  

If we can accurately estimate total duration from early signals, we can help guide users more intelligently.

### 1.3 Evaluation Setup

To evaluate the model fairly:

- **Train/Test Split:**  
  - Split **by userId**, so the **test set contains users not seen** during training.  
  - This simulates deploying the model to new users.

- **Metrics (Regression):**
  - **MAE (Mean Absolute Error)** – average absolute difference in minutes  
  - **RMSE (Root Mean Squared Error)** – penalizes large errors  
  - **R² (Coefficient of Determination)** – fraction of variance explained

- **Baselines:**
  1. **Global Mean Duration Baseline**  
     Always predict the **overall mean duration** from the training data.
  2. **Sport-specific Mean Baseline**  
     Predict the **mean duration per sport** (e.g., running vs cycling).

Any useful model should beat these simple baselines.


In [2]:
# =====================================
# Configuration and helper definitions
# =====================================

# Path to the EndoMondo HR dataset (adjust to your local file)
DATA_PATH = Path("data") / "endomondoHR.json"  # <-- CHANGE if needed

# Sampling interval between measurements, in seconds.
# Many processed EndoMondo datasets are resampled at 10-second intervals.
SAMPLING_INTERVAL_SECONDS = 10

# Early window length: how much of the workout we use as input
EARLY_WINDOW_MINUTES = 5
EARLY_STEPS = (EARLY_WINDOW_MINUTES * 60) // SAMPLING_INTERVAL_SECONDS


def safe_parse_sequence(x):
    """
    Ensure that a value is converted to a Python list.
    - If it's already a list, return as-is.
    - If it's a string representation of a list, parse with ast.literal_eval.
    - Otherwise, return an empty list.
    """
    if isinstance(x, list):
        return x
    if isinstance(x, (np.ndarray, pd.Series)):
        return list(x)
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return parsed
        except (ValueError, SyntaxError):
            pass
    # Fallback
    return []


def compute_duration_minutes(sequence_length):
    """
    Compute total workout duration in minutes from the sequence length
    and the known sampling interval.
    """
    duration_seconds = sequence_length * SAMPLING_INTERVAL_SECONDS
    return duration_seconds / 60.0


## 2. Exploratory Data Analysis (EDA) & Preprocessing

In this section I:

1. **Load** the EndoMondo dataset from a JSON-lines file.  
2. **Clean** and standardize the data:
   - Parse sequential features (`heart_rate`, `speed`) into Python lists  
   - Remove workouts with missing or empty sequences  
   - Derive total duration as the target variable  
3. Perform **exploratory data analysis (EDA)**:
   - Distribution of sports  
   - Distribution of workout durations  
   - Example time series for heart rate and speed  
4. **Process categorical and sequential features** into a tabular format suitable for machine learning.


In [3]:
# ======================
# Load the raw JSON data
# ======================

# Each line in the JSON file is assumed to be a Python dict representing one workout.
workouts = []

with open(DATA_PATH, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        # ast.literal_eval is safer than eval and sufficient for typical course datasets.
        try:
            workouts.append(ast.literal_eval(line))
        except Exception as e:
            # If a line fails to parse, we skip it (and could log this in a real project)
            continue

print(f"Loaded {len(workouts)} raw workout records.")


KeyboardInterrupt: 

In [ ]:
# =========================================
# Convert list of dicts to a pandas DataFrame
# =========================================

raw_df = pd.DataFrame(workouts)
print("Columns:", raw_df.columns.tolist())
raw_df.head()


In [ ]:
# ======================
# Basic sanity filtering
# ======================

# Keep only rows that at least have userId, sport, and heart_rate + speed info
required_cols = ["userId", "sport", "heart_rate"]
for col in required_cols:
    if col not in raw_df.columns:
        raise ValueError(f"Required column '{col}' not found. "
                         "Please adjust the code to match your dataset's schema.")

# Some datasets use 'derived_speed', others 'speed'
speed_col = "derived_speed" if "derived_speed" in raw_df.columns else "speed"
if speed_col not in raw_df.columns:
    raise ValueError("Could not find 'derived_speed' or 'speed' column. "
                     "Please update 'speed_col' to your dataset.")

# Drop rows with missing userId or sport
df = raw_df.dropna(subset=["userId", "sport"]).copy()

# Ensure we have heart_rate and speed sequences
df = df[(df["heart_rate"].notnull()) & (df[speed_col].notnull())].copy()
print(f"Remaining workouts after basic filtering: {len(df)}")


In [ ]:
# =======================================
# Parse heart_rate and speed into lists
# =======================================

df["heart_rate_seq"] = df["heart_rate"].apply(safe_parse_sequence)
df["speed_seq"] = df[speed_col].apply(safe_parse_sequence)

# Filter out workouts with empty or very short sequences
df["seq_len"] = df["heart_rate_seq"].apply(len)
df = df[df["seq_len"] >= EARLY_STEPS].copy()

print(f"Workouts with sufficiently long sequences (>= {EARLY_STEPS} steps): {len(df)}")

# ============================
# Compute target: duration_min
# ============================

df["duration_min"] = df["seq_len"].apply(compute_duration_minutes)
df["duration_min"].describe()


### 2.1 EDA: Sports and Duration Distributions

Before engineering features, I explore the basic properties of the dataset:

- **Sport distribution** – which activities are most common?
- **Duration distribution** – how long do workouts typically last?

Understanding these distributions helps:
- Identify potential **class imbalance** by sport
- Decide how to **filter outliers** (e.g., extremely short or long sessions)
- Interpret model errors relative to typical workout lengths


In [ ]:
# ===================
# Sport distribution
# ===================

sport_counts = df["sport"].value_counts().sort_values(ascending=False)
print("Number of unique sports:", df["sport"].nunique())
sport_counts.head(10)


In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(x=sport_counts.head(10).index, y=sport_counts.head(10).values)
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Sports by Workout Count")
plt.xlabel("Sport")
plt.ylabel("Number of workouts")
plt.tight_layout()
plt.show()


In [ ]:
# =======================
# Duration distribution
# =======================

plt.figure(figsize=(8, 4))
sns.histplot(df["duration_min"], bins=50, kde=True)
plt.title("Distribution of Workout Duration (minutes)")
plt.xlabel("Duration (minutes)")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


**Observations:**

- The **sport distribution** plot shows which activities dominate the dataset (for example, running, cycling, walking).  
- The **duration distribution** often shows a cluster around common workout times (e.g., 20–60 minutes), with fewer very short or very long sessions.

These patterns inform:
- How we interpret model errors (e.g., a 10-minute error is large for a 20-minute run but less critical for a 3-hour ride).  
- Whether we should **filter outliers** to create a more homogeneous prediction task.


In [ ]:
# ========================================
# (Optional) Filter out extreme durations
# ========================================

# For robustness, remove extremely short (< 5 min) and extremely long (> 3 hours) workouts
df = df[(df["duration_min"] >= 5) & (df["duration_min"] <= 180)].copy()
print(f"Workouts after duration filtering: {len(df)}")

# (Optional) focus on most common sports to simplify modeling
top_sports = sport_counts.head(5).index.tolist()
df = df[df["sport"].isin(top_sports)].copy()
print(f"Workouts after focusing on top 5 sports: {len(df)}")


In [ ]:
# =====================================
# Example heart-rate and speed time series
# =====================================

example_row = df.sample(1, random_state=RANDOM_STATE).iloc[0]

hr_seq = np.array(example_row["heart_rate_seq"])
sp_seq = np.array(example_row["speed_seq"])
time_minutes = np.arange(len(hr_seq)) * SAMPLING_INTERVAL_SECONDS / 60.0

plt.figure(figsize=(10, 4))
plt.plot(time_minutes, hr_seq, label="Heart rate")
plt.xlabel("Time (minutes)")
plt.ylabel("Heart rate (units)")
plt.title(f"Heart-rate over time for one {example_row['sport']} workout")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(time_minutes, sp_seq, label="Speed")
plt.xlabel("Time (minutes)")
plt.ylabel("Speed (units)")
plt.title(f"Speed over time for one {example_row['sport']} workout")
plt.legend()
plt.tight_layout()
plt.show()


### 2.2 EDA: Example Sequential Patterns

The example heart-rate and speed plots show the **sequential nature** of the data:

- Heart rate often starts lower and gradually increases as the user warms up.
- Speed may exhibit intervals of acceleration, steady cruising, and deceleration.

These patterns motivate the feature engineering strategy:
- Use **only the first 5 minutes** of these sequences (the "warm-up" period)
- Summarize them into **statistical features** (mean, variability, slope)  
This lets us plug sequential information into standard tabular ML models.


In [ ]:
# ==========================================
# Feature engineering from early sequences
# ==========================================

EARLY_STEPS = min(EARLY_STEPS, df["seq_len"].min())  # ensure we don't exceed shortest sequence

def extract_early_features(row):
    """
    Extract summary features from the first EARLY_STEPS of heart-rate and speed sequences.
    Also include simple metadata like sport and gender.
    """
    hr = np.array(row["heart_rate_seq"])
    sp = np.array(row["speed_seq"])

    hr_early = hr[:EARLY_STEPS]
    sp_early = sp[:EARLY_STEPS]

    feats = {}

    def add_stats(prefix, arr):
        arr = np.asarray(arr, dtype=float)
        feats[f"{prefix}_mean"] = np.nanmean(arr)
        feats[f"{prefix}_std"] = np.nanstd(arr)
        feats[f"{prefix}_min"] = np.nanmin(arr)
        feats[f"{prefix}_max"] = np.nanmax(arr)
        # simple slope: (last - first) / window length
        feats[f"{prefix}_slope"] = (arr[-1] - arr[0]) / max(len(arr), 1)

    add_stats("hr", hr_early)
    add_stats("sp", sp_early)

    # Metadata / categorical features
    feats["sport"] = row["sport"]
    feats["gender"] = row.get("gender", "unknown") if "gender" in df.columns else "unknown"
    feats["userId"] = row["userId"]

    # Target variable: total workout duration in minutes
    feats["duration_min"] = row["duration_min"]

    return pd.Series(feats)

features_df = df.apply(extract_early_features, axis=1)

print("Engineered feature columns:")
print(features_df.columns.tolist())
features_df.head()


### 2.3 Preprocessing & Train/Test Split

Now that we have engineered features, we:

1. **Handle missing values** in numeric features (e.g., from NaNs in early stats).  
2. Separate **features** (X) from the **target** (y = duration in minutes).  
3. Perform a **user-level train/test split**, so the test set contains users not seen during training.  

We also set up a **scikit-learn preprocessing pipeline** that:
- One-hot encodes categorical variables (`sport`, `gender`)
- Standardizes numerical features (heart-rate and speed statistics)


In [ ]:
# =======================================
# Handle missing values and define splits
# =======================================

# Drop rows with missing target
features_df = features_df.dropna(subset=["duration_min"]).copy()

# Identify feature types
target_col = "duration_min"
categorical_cols = ["sport", "gender"]
numeric_cols = [c for c in features_df.columns
                if c not in [target_col, "userId"] + categorical_cols]

# Simple imputation: fill any remaining NaNs in numeric features with column means
features_df[numeric_cols] = features_df[numeric_cols].fillna(features_df[numeric_cols].mean())

# Train/test split at user level
unique_users = features_df["userId"].unique()
train_users, test_users = train_test_split(
    unique_users, test_size=0.2, random_state=RANDOM_STATE
)

train_df = features_df[features_df["userId"].isin(train_users)].copy()
test_df = features_df[features_df["userId"].isin(test_users)].copy()

print("Train size:", train_df.shape, "| Test size:", test_df.shape)

X_train = train_df[categorical_cols + numeric_cols]
y_train = train_df[target_col]

X_test = test_df[categorical_cols + numeric_cols]
y_test = test_df[target_col]

# ============================
# Preprocessing with ColumnTransformer
# ============================

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numeric_cols),
    ]
)


## 3. Modeling – Baselines and Machine Learning Models

To assess how much predictive signal exists in early-workout features, I implement:

### 3.1 Baselines

1. **Global Mean Baseline**  
   Always predicts the **mean duration** from the training data.

2. **Sport-specific Mean Baseline**  
   For each workout, predicts the **average duration for that sport** based on the training set.  
   If a sport is unseen (rare due to our filtering), it falls back to the global mean.

These represent simple human-like heuristics (e.g., "runs are usually ~30 minutes").

### 3.2 Machine Learning Models

1. **Linear Regression**  
   - A simple, interpretable model.  
   - Assumes a linear relationship between features and workout duration.

2. **Random Forest Regressor**  
   - An ensemble of decision trees.  
   - Captures nonlinear interactions between features.  
   - Generally robust to outliers and mixed feature types.

Both models are implemented using **scikit-learn pipelines** with the same preprocessing steps for a fair comparison.


In [ ]:
# ===========================
# Baseline models (no fitting)
# ===========================

# Global mean duration baseline
global_mean_duration = y_train.mean()

# Sport-specific mean duration baseline
sport_mean_duration = train_df.groupby("sport")[target_col].mean().to_dict()


def baseline_global_mean(X):
    """Predict using the global mean duration."""
    return np.full(shape=(len(X),), fill_value=global_mean_duration)


def baseline_sport_mean(X):
    """
    Predict using mean duration per sport.
    Falls back to global mean if sport is unseen.
    """
    preds = []
    for _, row in X.iterrows():
        sp = row["sport"]
        preds.append(sport_mean_duration.get(sp, global_mean_duration))
    return np.array(preds)


In [ ]:
# ===============================
# Linear Regression Model
# ===============================

linreg_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LinearRegression())
])

linreg_pipeline.fit(X_train, y_train)

# ===============================
# Random Forest Regressor Model
# ===============================

rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=200,
        max_depth=None,
        n_jobs=-1,
        random_state=RANDOM_STATE
    ))
])

rf_pipeline.fit(X_train, y_train)


## 4. Evaluation – Metrics, Comparisons, and Interpretation

To evaluate each model, I use:

- **MAE (Mean Absolute Error)** – average absolute error in minutes  
- **RMSE (Root Mean Squared Error)** – penalizes large errors  
- **R² (Coefficient of Determination)** – proportion of variance explained  

I compare:

1. **Global Mean Baseline**  
2. **Sport Mean Baseline**  
3. **Linear Regression**  
4. **Random Forest Regressor**

We expect the machine learning models (especially Random Forest) to improve upon the baselines if early-workout features contain useful predictive information.


In [ ]:
# ==========================================
# Helper function to compute regression metrics
# ==========================================

def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}


# ==========================
# Evaluate all models
# ==========================

results = {}

# Baselines
y_pred_global = baseline_global_mean(X_test)
y_pred_sport = baseline_sport_mean(X_test)

results["Global mean"] = regression_metrics(y_test, y_pred_global)
results["Sport mean"] = regression_metrics(y_test, y_pred_sport)

# Linear Regression
y_pred_lin = linreg_pipeline.predict(X_test)
results["Linear regression"] = regression_metrics(y_test, y_pred_lin)

# Random Forest
y_pred_rf = rf_pipeline.predict(X_test)
results["Random forest"] = regression_metrics(y_test, y_pred_rf)

results_df = pd.DataFrame(results).T
results_df


In [ ]:
# ==========================
# Bar plots for MAE and RMSE
# ==========================

plt.figure(figsize=(8, 4))
sns.barplot(x=results_df.index, y=results_df["MAE"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("MAE (minutes)")
plt.title("Model Comparison: MAE on Test Set")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
sns.barplot(x=results_df.index, y=results_df["RMSE"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("RMSE (minutes)")
plt.title("Model Comparison: RMSE on Test Set")
plt.tight_layout()
plt.show()


In [ ]:
# ==========================
# Predicted vs true for Random Forest
# ==========================

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred_rf, alpha=0.3)
lims = [min(y_test.min(), y_pred_rf.min()), max(y_test.max(), y_pred_rf.max())]
plt.plot(lims, lims, "r--", linewidth=1, label="Perfect prediction")
plt.xlim(lims)
plt.ylim(lims)
plt.xlabel("True duration (minutes)")
plt.ylabel("Predicted duration (minutes)")
plt.title("Random Forest: Predicted vs. True Duration")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ==========================
# Error analysis by sport
# ==========================

test_df = test_df.copy()
test_df["pred_rf"] = y_pred_rf
test_df["abs_error"] = (test_df["pred_rf"] - test_df["duration_min"]).abs()

sport_error = test_df.groupby("sport")["abs_error"].mean().sort_values()

plt.figure(figsize=(8, 4))
sns.barplot(x=sport_error.index, y=sport_error.values)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Mean absolute error (minutes)")
plt.title("Random Forest MAE by Sport")
plt.tight_layout()
plt.show()

sport_error


### 4.1 Quantitative Results

The **results table** and **bar plots** show performance for:

- Global Mean  
- Sport Mean  
- Linear Regression  
- Random Forest  

Typical patterns (your exact numbers will depend on the data and filtering):

- The **Global Mean** baseline has the **largest errors**, since it ignores any sport or early-workout information.
- The **Sport Mean** baseline improves, reflecting that different sports have different typical durations.
- **Linear Regression** further reduces MAE and RMSE, showing that early heart-rate and speed statistics contain predictive signal.
- The **Random Forest** usually achieves the **lowest MAE/RMSE and highest R²**, indicating that nonlinear relationships between features and duration are important.

### 4.2 Predicted vs True Durations

The scatter plot for the Random Forest model:

- Points clustered around the diagonal indicate **good predictions**.
- Deviations, especially for very long workouts, show where the model struggles.
- This suggests that **extreme sessions** are harder to predict from early signals alone.

### 4.3 Error by Sport

The error-by-sport bar chart shows:

- Some sports (e.g., steady-state activities like cycling) may have **lower average errors**.
- Others (e.g., more variable or less frequent sports) may have **higher errors**.

This highlights the importance of **data quantity** and **variability** per sport, and suggests directions for more specialized or sport-specific models.


## 5. Discussion of Related Work and Insights

### 5.1 Related Work (paraphrased)

There is a growing body of work using **wearable sensor data** and **fitness tracking logs** for prediction and recommendation:

- Some studies use **EndoMondo/Endomondo-like data** to build **personalized fitness recommenders**, forecasting how heart rate and speed will evolve over a workout and suggesting appropriate future activities.
- Other work uses **neural networks and dynamic models** to predict **heart-rate responses** under different workloads, with applications to training optimization and health monitoring.
- Research on **cyclist and runner performance modeling** often employs **sequence models** (e.g., RNNs, LSTMs) to capture temporal dynamics in power, speed, and heart-rate data.

These projects typically focus on **fine-grained trajectory prediction** or **performance estimation**, whereas my project simplifies the problem to predicting a **single scalar outcome**: total workout duration.

### 5.2 Comparison to My Approach

Compared to these more complex setups:

- I use a **simpler feature-based approach**, summarizing early sequences into a small number of statistics (mean, variability, slope).
- Instead of deep sequence models, I use **tabular ML models** (linear regression, random forest) that are easy to train, interpret, and run quickly on standard hardware.
- My evaluation with **user-level train/test splits** reflects an interest in **generalizing to new users**, similar to deployment scenarios in real fitness apps.

Despite its simplicity, the model:

- Significantly outperforms naive baselines.
- Demonstrates that **early heart-rate and speed patterns** contain useful information about total workout duration.

### 5.3 Limitations and Future Directions

Limitations:

- Early features are relatively coarse; they may discard useful temporal information (e.g., exact shape of the HR curve).
- I do not explicitly model **route, elevation, or weather** here, even though they are available in some EndoMondo datasets.
- Aggregating across all users may ignore **individual differences**, which could matter for personalized predictions.

Potential extensions:

- Replace summary statistics with **full sequence models** (RNNs, 1D CNNs) to capture more detailed dynamics.
- Incorporate **route and elevation** to better understand how terrain affects duration.
- Train **personalized or multi-task models** that explicitly account for user-specific behavior.


## 6. Conclusion

In this notebook, I:

1. Defined a **regression task**: predicting total workout duration from the **first 5 minutes** of heart-rate and speed data plus basic context.  
2. Performed **EDA** on sports and duration distributions, as well as example time-series plots.  
3. Engineered features from sequential data (early-window statistics) and processed categorical variables with **one-hot encoding**.  
4. Built and evaluated multiple models:
   - Simple baselines (global mean, sport-specific mean)
   - Machine learning models (linear regression, random forest)
5. Compared performance using **MAE, RMSE, and R²**, including **error analysis by sport**.
6. Discussed **related work** and positioned this project within the broader literature on fitness and wearable data modeling.

The results show that early sensor data contains substantial predictive signal about total workout duration, especially when modeled with a non-linear ensemble like a random forest.

---

## 7. How to Export for Submission (HTML)

Once this notebook runs top-to-bottom without errors and all plots are visible:

1. **Save the notebook** as `endomondo_final_project.ipynb`.
2. From a terminal or command prompt in the same directory, run:

   ```bash
   jupyter nbconvert --to html endomondo_final_project.ipynb
